# Chattbot med RAG (Retrieval-Augmented Generation)

**Datum:** 27 maj 2025  
**Student:** Hani Abraksiea  
**Kurs:** Data Scientist – EC Utbildning  
**Syfte:**  
Syftet med detta projekt är att bygga en chattbot som använder RAG-teknik för att svara på frågor utifrån innehåll i en PDF-fil. Modellen begränsar sina svar till det material som finns i dokumentet, och svarar "Det vet jag inte" om informationen inte finns.

Målet är att visa teknisk förståelse för RAG, samt kunna resonera kring användning, utmaningar och etiska perspektiv.


In [1]:
import os
import google.generativeai as genai
from dotenv import load_dotenv
import numpy as np
from pypdf import PdfReader

# Ladda API-nyckel från .env
load_dotenv()
genai.configure(api_key=os.getenv("API_KEY"))

c:\Users\hani7\OneDrive\Dokument\Data_scientist_EC_utbildning\07_dl\Kunskapkontrol_2\genai_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Läs in PDF (Magisteruppsats)
reader = PdfReader("Magisteruppsats.pdf")
text = "".join([page.extract_text() for page in reader.pages])

# Dela upp i chunks
chunk_size = 1000
overlap = 200
chunks = []
for i in range(0, len(text), chunk_size - overlap):
    chunks.append(text[i:i + chunk_size])

print(f"Antal chunks: {len(chunks)}")


Antal chunks: 254


In [3]:
embedding_model = "models/embedding-001"

def get_embeddings(text_list):
    return [genai.embed_content(
        model=embedding_model,
        content=txt,
        task_type="retrieval_document"
    )["embedding"] for txt in text_list]

chunk_embeddings = get_embeddings(chunks)

In [8]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def semantic_search(query, chunks, embeddings, k=5):
    query_embedding = genai.embed_content(
        model=embedding_model,
        content=query,
        task_type="retrieval_query"
    )["embedding"]
    
    scores = [(i, cosine_similarity(query_embedding, emb)) for i, emb in enumerate(embeddings)]
    scores.sort(key=lambda x: x[1], reverse=True)
    top_chunks = [chunks[i] for i, _ in scores[:k]]
    
    return "\n".join(top_chunks)

In [9]:
model = genai.GenerativeModel("gemini-1.5-flash")

def generate_answer(query):
    context = semantic_search(query, chunks, chunk_embeddings)
    prompt = f"""
Du är en hjälpsam assistent. Svara bara baserat på följande kontext:
{context}

Fråga: {query}
Om svaret inte finns i kontexten, säg "Det vet jag inte."
"""
    response = model.generate_content(prompt)
    return response.text

In [15]:
test_cases = [
    {
        "question": "Vilken typ av intervjuer användes i studien?",
        "expected_contains": "semistrukturerade"
    },
    {
        "question": "Vilken kommun är studien baserad på?",
        "expected_contains": "Nyköpings"
    },
    {
        "question": "Vilket arbetssätt handlar uppsatsen om?",
        "expected_contains": "Agila arbetssätt inom IT-avdelningen i Nyköpings kommun."
    },
    {
        "question": "Vilket år grundades kommunen?",
        "expected_contains": "Det vet jag inte" # Ej med i dokumentet
    }
]

In [16]:
for case in test_cases:
    print("Fråga:", case["question"])
    answer = generate_answer(case["question"])
    print("Svar:", answer)
    print("Godkänt?" , "✓" if case["expected_contains"].lower() in answer.lower() else "✗")
    print("-" * 40)


Fråga: Vilken typ av intervjuer användes i studien?
Svar: Studien använde semistrukturerade intervjuer.

Godkänt? ✓
----------------------------------------
Fråga: Vilken kommun är studien baserad på?
Svar: Studien är baserad på Nyköpings kommun.

Godkänt? ✓
----------------------------------------
Fråga: Vilket arbetssätt handlar uppsatsen om?
Svar: Uppsatsen handlar om agila arbetssätt inom IT-avdelningen i Nyköpings kommun.

Godkänt? ✓
----------------------------------------
Fråga: Vilket år grundades kommunen?
Svar: Det vet jag inte.

Godkänt? ✓
----------------------------------------


## Reflektion: Användning, möjligheter och utmaningar

Chattboten använder Retrieval-Augmented Generation (RAG), en metod där en språkmodell först hämtar relevant information från en specifik källa – i detta fall min magisteruppsats – och därefter genererar ett svar baserat på den. Detta tillvägagångssätt minskar risken för att modellen "hittar på" information och ger mer kontextmedvetna och korrekta svar.

### Möjlig användning:
- Forskare som snabbt vill ställa frågor om innehållet i sina egna studier
- Studenter som vill förstå, citera eller referera akademiska texter korrekt
- Journalister som analyserar större dokument eller rapporter
- Myndigheter eller organisationer som vill erbjuda sökbarhet i interna dokument

### Etiska och tekniska utmaningar:
- AI:n kan ändå ge osäkra eller vaga svar om kontexten är otydlig eller ofullständig
- RAG-system måste hantera upphovsrätt korrekt om externa dokument används
- Det finns risk för att användaren tolkar svaren som absoluta sanningar
- Om systemet används med känsliga eller personliga dokument måste GDPR och datasäkerhet beaktas

### Affärspotential:
- Ett kraftfullt verktyg för universitet, bibliotek eller forskningsdatabaser
- Möjlighet att integrera i intranät, dokumentarkiv eller e-tjänster
- Kundserviceverktyg för organisationer med omfattande PDF-manualer eller policyer

### Slutsats:
Detta projekt visar att RAG-teknik kan skapa mer tillförlitliga och ansvarsfulla AI-assistenter. Genom att begränsa modellens svar till specifika dokument blir interaktionen både mer trovärdig och användbar – särskilt i kunskapsintensiva miljöer.
